# <center> Regularized Linear Regression from Scratch </center> <br> 
<center>Prepared by Wyatt Walsh</center>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Configure-Environment-and-Load-Libraries/Data" data-toc-modified-id="Configure-Environment-and-Load-Libraries/Data-1">Configure Environment and Load Libraries/Data</a></span></li><li><span><a href="#Split-data-into-training-and-testing-sets" data-toc-modified-id="Split-data-into-training-and-testing-sets-2">Split data into training and testing sets</a></span></li><li><span><a href="#Data-Cleaning-&amp;-Exploratory-Data-Analysis" data-toc-modified-id="Data-Cleaning-&amp;-Exploratory-Data-Analysis-3">Data Cleaning &amp; Exploratory Data Analysis</a></span></li></ul></div>

## Configure Environment and Load Libraries/Data
First, necessary libraries are imported and the notebook is configured for future plotting

In [1]:
# import tensorflow.experimental.numpy as tnp
import numpy as np
from math import isclose
import pandas as pd 
import time

from src import utilities as utils
from src import linear_regression as lr
from src import cross_validation as cv

from sklearn.linear_model import LinearRegression as skOLS
from sklearn.linear_model import RidgeCV
# %matplotlib inline
# from mpl_toolkits.mplot3d import Axes3D
# import matplotlib.pyplot as plt

In [2]:
wine_data_raw = pd.read_csv('data/winequality-red.csv', sep = ';')
wine_data_raw.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


## Split data into training and testing sets
Before further modeling, the data should be split into training and test sets so that the accuracy of final models can be better assessed.

In [3]:
train_proportion = 0.8
train,test = utils.test_train_split(wine_data_raw, train_proportion)
train_vals, test_vals = train.to_numpy(), test.to_numpy()
X_train = train_vals[:, 0:-1]
y_train = train_vals[:, [-1]]
X_test = test_vals[:, 0:-1]
y_test = test_vals[:, [-1]]
display(train.head()), display(test.head())
print('With a training proportion of {}, \
there are {} rows in the training set and {} rows in the test set'.format(train_proportion, len(train), len(test)))

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.7,0.56,0.08,2.50,0.114,14.0,46.0,0.9971,3.24,0.66,9.6,6
1,7.8,0.50,0.17,1.60,0.082,21.0,102.0,0.9960,3.39,0.48,9.5,5
2,10.7,0.67,0.22,2.70,0.107,17.0,34.0,1.0004,3.28,0.98,9.9,6
3,8.5,0.46,0.31,2.25,0.078,32.0,58.0,0.9980,3.33,0.54,9.8,5
4,6.7,0.46,0.24,1.70,0.077,18.0,34.0,0.9948,3.39,0.60,10.6,6


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,8.0,0.28,0.44,1.8,0.081,28.0,68.0,0.99501,3.36,0.66,11.2,5
1,7.0,0.50,0.14,1.8,0.078,10.0,23.0,0.99636,3.53,0.61,10.4,5
2,6.0,0.50,0.00,1.4,0.057,15.0,26.0,0.99448,3.36,0.45,9.5,5
3,8.0,0.59,0.05,2.0,0.089,12.0,32.0,0.99735,3.36,0.61,10.0,5
4,6.5,0.53,0.06,2.0,0.063,29.0,44.0,0.99489,3.38,0.83,10.3,6


With a training proportion of 0.8, there are 1279 rows in the training set and 320 rows in the test set


## Data Cleaning & Exploratory Data Analysis
First, variance inflation factor (VIF) calculations are made to examine possible multicollinearity of the predictors by iteratively removing the highest scoring predictor with a score above 5.

In [4]:
utils.VIF(X_train)

('Low Multicollinearity!',
 array([ 3.07389133,  1.75308848,  3.05862454,  1.10142866,  1.46048231,
         1.97929531,  2.23184107, -0.00517207,  2.26784033,  1.34956851,
         1.32223964]))

With the data pre-processing and evaluation step completing, computing different estimates of the model parameters can be conducted. 

Both Ordinary Least Squares (OLS) and Ridge Regression have closed form solutions for the values of the parameters that minimize the squared loss so those equations are solved and computed! 

In [5]:
start = time.time()
ols = lr.ols(X_train, y_train)
ols_error = utils.get_error(ols, test_vals)
end = time.time()
times_OLS = end-start
print('My OLS Function\'s Runtime: ', end - start)

start = time.time()
OLS_fitted_sklearn = skOLS(fit_intercept= True).fit(X_train, y_train)
OLS_predictions_sklearn = OLS_fitted_sklearn.predict(test.values[:, 0:-1])
OLS_error_sklearn = np.linalg.norm(test.values[:,-1]-OLS_predictions_sklearn)**2
end = time.time()
times_OLS = np.append(times_OLS, end - start)
print('Scikit-learn\'s OLS Function Runtime: ', end - start)

print('Are the error values close?', " ", isclose(ols_error,OLS_error_sklearn))

OLS_sklearn_row = np.append([times_OLS[1], OLS_error_sklearn, OLS_fitted_sklearn.intercept_], OLS_fitted_sklearn.coef_)
ols_row = np.append(np.append(times_OLS[0], ols_error), ols)
OLS_df = pd.DataFrame(np.vstack((ols_row, OLS_sklearn_row)), columns = np.append(['Runtime (s)','Error','Y-Intercept'],train.columns[0:-1]))
OLS_df.index = ['My Function', "Scikit-Learn's Function"]
OLS_df

My OLS Function's Runtime:  0.0017201900482177734
Scikit-learn's OLS Function Runtime:  0.00404810905456543
Are the error values close?   True


/Users/wyattowalsh/miniconda3/envs/regularized-regression-from-scratch/lib/python3.8/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


,Runtime (s),Error,Y-Intercept,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
My Function,0.00172019,85308.6,22.4645,0.00653813,-0.952018,-0.0861438,0.00958988,-1.92368,0.00493009,-0.00364213,-17.4434,-0.659216,0.985946,0.269186
Scikit-Learn's Function,0.00404811,85308.6,[22.46453362811144],0.00653813,-0.952018,-0.0861438,0.00958988,-1.92368,0.00493009,-0.00364213,-17.4434,-0.659216,0.985946,0.269186


In [8]:
start = time.time()
ridge = cv.ridge(X_train, y_train)
ridge_error = utils.get_error(ridge[3], test_vals)
end = time.time()
times_ridge = end-start
print('My Ridge Function\'s Runtime: ', times_ridge)

start = time.time()
ridge_fitted_sklearn = RidgeCV(alphas = (0.1,1,100), fit_intercept= True, cv = 5).fit(train_vals[:,0:-1],
                                                                               train_vals[:,-1])
ridge_predictions_sklearn = ridge_fitted_sklearn.predict(test_vals[:, 0:-1])
ridge_error_sklearn = np.linalg.norm(test.values[:,-1]-
                                     ridge_predictions_sklearn)**2
end = time.time()
times_ridge = np.append(times_ridge, end - start)
print('Scikit-learn\'s Ridge Function Runtime: ', end - start)

print('Are the error values close?', " ", isclose(ridge_error,ridge_error_sklearn) or ridge_error < ridge_error_sklearn)

ridge_sklearn_row = np.append([times_ridge[1], ridge_error_sklearn, ridge_fitted_sklearn.intercept_], 
                              ridge_fitted_sklearn.coef_)
ridge_row = np.append(np.append(times_ridge[0], ridge_error), ridge[3])
ridge_df = pd.DataFrame(np.vstack((ridge_row, ridge_sklearn_row)), columns = np.append(['Runtime (s)','Error','Y-Intercept'],train.columns[0:-1]))
ridge_df.index = ['My Function', "Scikit-Learn's Function"]
ridge_df

My Ridge Function's Runtime:  0.1403791904449463
Scikit-learn's Ridge Function Runtime:  0.03422999382019043
Are the error values close?   True


,Runtime (s),Error,Y-Intercept,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
My Function,0.140379,111.674604,4.999346,-0.009309,-0.998126,-0.021753,-0.001947,-1.357499,0.004224,-0.003436,-0.025877,-0.605643,0.839102,0.282051
Scikit-Learn's Function,0.034230,111.876011,4.971814,-0.002578,-0.982513,-0.104107,0.001340,-1.289327,0.005073,-0.003564,-0.017864,-0.653298,0.864480,0.292262


In [10]:
df = np.hstack((X_train, y_train))
indices = np.arange(len(y_train))
np.random.shuffle(indices)
sets = np.array_split(indices, 5)
fn = lambda index: df[sets[index],:]
ls = [fn(i) for i in np.arange(5)]

In [8]:
np.mean(np.array(cv.ridge(X_train, y_train)),0).argmin()

45

In [5]:
np.shape(X_train)

(1279, 11)

In [7]:
ridge_fitted_sklearn.coef_

array([-0.00442999, -0.98084872, -0.10239602,  0.0016772 , -1.45279947,
        0.00506085, -0.00359124, -0.02448573, -0.67885945,  0.88997452,
        0.29059683])

\begin{align}
\begin{bmatrix}
    \hat{\beta}_0\\
    \hat{\beta}\\
\end{bmatrix}
&=
\arg\min_{\beta_0,\beta} \left\Vert\mathbf{Y}-\begin{bmatrix} \mathbf{1_n} & \mathbf{X} \end{bmatrix}\begin{bmatrix}
    \hat{\beta}_0\\
    \hat{\beta}\\
\end{bmatrix}\right\Vert^2 _2
\\
\begin{bmatrix}
    \hat{\beta}\\
\end{bmatrix}
&=
{(\mathbf{X'}\mathbf{X}})^{-1} (\mathbf{X'}\mathbf{Y})
\end{align}

Lasso Regression and the Elastic Net however do not have closed form solutions due to the nature of the L1 norm penalization term. This means that discrete optimization methods are needed in order to compute the regression estimates using these regularization techniques. Currently one of the best algorithms to accomplish this is Coordinate Descent that incorporates warm starts and covariance/naive updates.

Now, in order to find the correct tuning parameters for Ridge, Lasso, and the Elastic Net K-fold Cross Validation is used. We use a k = 10 since our data set is only 1599 entries, and make our grid spacing on a log scale for lambda and a log scale for alpha (in the case of the Elastic Net)

In [ ]:
start = time.time()
ridge = cv.ridge(train.values)
ridge_error = utils.get_error(ridge[3], test)
end = time.time()
times_ridge = end-start
print('My Ridge Function\'s Runtime: ', times_ridge)


In [ ]:
ridge = cv.cross_validation_error(train,regress.ridge,100,5)
lasso = cv.cross_validation_error(train,regress.lasso,100,5)
# elastic = cross_validation(wine_data,10,100, Type = 'elastic net')

In [ ]:
y_hat = np.dot(test.values[:,0:-1],ols[1:])+ols[0]
error = np.linalg.norm(test.values[:,-1]-y_hat)**2
error

In [ ]:
lasso['Model Parameters'][0,1]

In [ ]:
train

In [ ]:
pd.DataFrame(index = ["OLS","Ridge","Lasso","Elastic Net"], 
              data = ({"Cross Validation Error": [ols_est,ridge[1],
                                                  lasso[1],elastic[2]]}))

In [ ]:
indice_alpha = np.where(elastic[5] == elastic[1])[0][0]
indice_lambda = np.where(elastic[4] == elastic[0])[0][0]
columns = np.append('Intercept', wine_data.columns.values[0:-1])
index = ["OLS","Ridge","Lasso standardized","Lasso unstandardized",\
         "Elastic Net standardized","Elastic Net unstandardized"]
data = np.append(np.append(np.append(
    np.append(np.append([ols(wine_data)], \ [ridge_regression(
        wine_data,ridge[0])],0), \ [lasso_regression_covar(
        wine_data,lasso[0])[0]],0),\ [unstandardize(
        lasso_regression_covar(wine_data,lasso[0])[0])],0),\
                 [elastic[-1][indice_alpha,:][indice_lambda,:]],0),\
                 [unstandardize(
    elastic[-1][indice_alpha,:][indice_lambda,:])],0)

pd.DataFrame(data = data, index = index, columns = columns)

In [ ]:
#Plot tuning parameters versus error and tuning parameters 
#versus coefficient value
fig = plt.figure(figsize=(20*1.5,9*1.5))
fig.tight_layout()

rid = fig.add_subplot(211)
rid.scatter(ridge[3], ridge[2])
rid.plot(ridge[3], ridge[2])
rid.set_xscale('log')
rid.set_xlabel('Log Lambda')
rid.set_ylabel("Cross Validation Error")
rid.set_title('Cross Validation Error for Ridge')

rid_coeff = fig.add_subplot(212)
rid_coeff.set_xlabel('Log Lambda')
rid_coeff.set_ylabel("Coefficients")
rid_coeff.set_title("Coefficient Path for Ridge")
for i in np.arange(1,12):
    rid_coeff.plot(ridge[3],ridge[4][:,i-1])
    rid_coeff.set_xscale('log')



In [ ]:
#Plot tuning parameters versus error and tuning parameters 
#versus coefficient value
fig2 = plt.figure(figsize=(20*1.5,9*1.5))
fig2.tight_layout()

las = fig2.add_subplot(211)
las.scatter(lasso[3], lasso[2])
las.plot(lasso[3], lasso[2])
las.set_xscale('log')
las.set_xlabel('Log Lambda')
las.set_ylabel("Cross Validation Error")
las.set_title('Cross Validation Error for Lasso')

las_coeff = fig2.add_subplot(212)
las_coeff.set_xlabel('Log Lambda')
las_coeff.set_ylabel("Coefficients")
las_coeff.set_title("Coefficient Path for Lasso")
for i in np.arange(1,12):
    las_coeff.plot(lasso[3],lasso[4][:,i])
    las_coeff.set_xscale('log')
    

In [ ]:
fig4,axs = plt.subplots(6,1,figsize = (25*1.5,30*1.5))

axs = axs.ravel()
for i in np.arange(1,7):
    X = elastic[4][int(99/(7-i)),:]
    Y = elastic[3][int(99/(7-i)),:]
    alpha = elastic[5][int(99/(7-i))]
    axs[i-1].plot(X,Y)
    axs[i-1].set_xscale('log')
    axs[i-1].set_xlabel('Log Lambda')
    axs[i-1].set_ylabel("Cross Validation Error")
    axs[i-1].set_title('Cross Validation Error for alpha = %f' %alpha)

In [ ]:
fig5 = plt.figure(figsize=(15,15))

elas_coeff = fig5.add_subplot(212)
elas_coeff.set_xlabel('Log Lambda')
elas_coeff.set_ylabel("Coefficients")
elas_coeff.set_title("Coefficient Path for Elastic Net")

indice = np.where(elastic[5] == elastic[1])[0][0]
for i in np.arange(1,12):
    elas_coeff.plot(elastic[4][indice,:],elastic[-1][indice,:][:,i])
    elas_coeff.set_xscale('log')